# SageMaker Neo for Build-in Object Detection

Because build-in object detection does not save network graph information such as ResNet50 or VGG16, the model only saves the parameters of weight. We need to add network graph information and save the checkpoints. After do so, it can use SageMaker Neo or convert it into ONNX.

In [1]:
%%bash
git clone https://github.com/apache/incubator-mxnet

Cloning into 'incubator-mxnet'...


In [2]:
import boto3
sm = boto3.client('sagemaker')
response = sm.describe_training_job(TrainingJobName='object-detection-2020-04-22-09-25-36-709')
model_path = response['ModelArtifacts']['S3ModelArtifacts']
model_path

's3://beyoung-sagemaker/coco-object-detection-20200422/output/object-detection-2020-04-22-09-25-36-709/output/model.tar.gz'

## Copy SageMaker Training job model to local notebook

cp model_algo_1-xxxxx both json and para to model folder, incubator-mxnet/example/ssd/model/

In [5]:
!aws s3 cp {model_path} ./model.tar.gz
!tar -xvf ./model.tar.gz -C ./incubator-mxnet/example/ssd/model/
!ls ./incubator-mxnet/example/ssd/model/

download: s3://beyoung-sagemaker/coco-object-detection-20200422/output/object-detection-2020-04-22-09-25-36-709/output/model.tar.gz to ./model.tar.gz
hyperparams.json
model_algo_1-0000.params
model_algo_1-symbol.json
hyperparams.json  model_algo_1-0000.params  model_algo_1-symbol.json  README.md


## Load model as checkpoint file with network information

In [6]:
# change notebook working directory
%cd ./incubator-mxnet/example/ssd

/home/ec2-user/SageMaker/myAWSStudyBlog/sagemaker-image-classification-detection/6-SageMaker-Neo/incubator-mxnet/example/ssd


In [7]:
#deploy model as checkpoint format
!python deploy.py --network resnet50 --prefix model/model_algo_1 --num-class 80 --data-shape 512

[06:54:16] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v1.4.1. Attempting to upgrade...
[06:54:16] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
Saved model: model/deploy_model_algo_1-0000.params
Saved symbol: model/deploy_model_algo_1-symbol.json


## tar model file

In [13]:
!cd ./model;tar cvfz sm-detection-model.tar.gz deploy_model_algo_1-symbol.json deploy_model_algo_1-0000.params

deploy_model_algo_1-symbol.json
deploy_model_algo_1-0000.params


In [14]:
!ls ./model

deploy_model_algo_1-0000.params  model_algo_1-symbol.json
deploy_model_algo_1-symbol.json  model.tar.gz
hyperparams.json		 README.md
model_algo_1-0000.params	 sm-detection-model.tar.gz


In [20]:
source_uri = 's3://beyoung-sagemaker/neo/sm-detection-model.tar.gz'
!aws s3 cp ./model/sm-detection-model.tar.gz {source_uri}

upload: model/sm-detection-model.tar.gz to s3://beyoung-sagemaker/neo/sm-detection-model.tar.gz


## SageMaker compiling job

In [21]:
from datetime import datetime
import sagemaker

compiling_job = datetime.now().strftime("compiling-detection-%Y-%m-%m-%H-%M")
role=sagemaker.get_execution_role()
output_uri = 's3://beyoung-sagemaker/neo/output'
role

'arn:aws:iam::476271697919:role/service-role/AmazonSageMaker-ExecutionRole-20200427T094649'

In [23]:
response = sm.create_compilation_job(
    CompilationJobName=compiling_job,
    RoleArn=role,
    InputConfig={
        'S3Uri': source_uri,
        'DataInputConfig': '{"data":[1,3,512,512]}',
        'Framework': 'MXNET'
    },
    OutputConfig={
        'S3OutputLocation': output_uri,
        'TargetDevice': 'ml_c5'
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900,
        'MaxWaitTimeInSeconds': 900
    }
)

In [24]:
response

{'CompilationJobArn': 'arn:aws:sagemaker:us-west-2:476271697919:compilation-job/compiling-detection-2020-05-05-07-51',
 'ResponseMetadata': {'RequestId': 'c47c2af6-759b-4506-9f17-66f49b16beab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c47c2af6-759b-4506-9f17-66f49b16beab',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '117',
   'date': 'Fri, 22 May 2020 07:54:43 GMT'},
  'RetryAttempts': 0}}